In [1]:
import json
import GPy
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from os.path import join, splitext
from os import listdir

In [2]:
data_path = '/home/rodrigo/Documents/curvas-de-luz-EL4106/OGLE_trainingDB/'
sub_class = 'EB/EC'

In [3]:
lc_list = [splitext(file)[0] for file in sorted(listdir(join(data_path, sub_class))) if splitext(file)[1] == '.dat']

In [4]:
print(len(lc_list))
file = open('database.csv','a')
file.write("phi - magnitude - class \n")
def smooth(x, window_len=11, window='hanning'):
    if window_len < 3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='valid')
    return y    

721


In [5]:
for lc_index in range(len(lc_list)):
    
    data = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.time'))
    period = np.loadtxt(join(data_path, sub_class, lc_list[lc_index]+'.dat'))
    mjd, mag_r, err_r, mag_b, err_b = data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4]
    """
    mjd_b = mjd
    mjd_r = mjd
    """
    mjd_b = mjd[err_b < 9.999]
    mjd_r = mjd[err_r < 9.999]
    mag_b = mag_b[err_b < 9.999]
    mag_r = mag_r[err_r < 9.999]
    err_b = err_b[err_b < 9.999]
    err_r = err_r[err_r < 9.999]

    mask = err_b < np.mean(err_b) + 3*np.std(err_b)
    mjd_b = mjd_b[mask]
    mag_b = mag_b[mask]
    err_b = err_b[mask]
    mask = err_r < np.mean(err_r) + 3*np.std(err_r)
    mjd_r = mjd_r[mask]
    mag_r = mag_r[mask]
    err_r = err_r[mask]
    
    phi = np.mod(mjd_b, period)/period
    I = np.argsort(phi)
    phi_rs = np.linspace(0, 1, num=200)
    mag_b_rs = np.interp(phi_rs, phi[I], mag_b[I])
    

    kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m = GPy.models.GPHeteroscedasticRegression(phi[:, np.newaxis], mag_b[:, np.newaxis], kernel)
    m['.*het_Gauss.variance'] = err_b[:, np.newaxis]**2 
    m.het_Gauss.variance.fix() 
    #m.het_Gauss.period.fix() 
    m.optimize()

    m.optimize(messages=True)
    m.optimize_restarts(num_restarts = 5)
    mu, var = m._raw_predict(phi_rs[:, np.newaxis])
    # File writer
    file.write(json.dumps(phi_rs.tolist())+" - "+json.dumps(mu[:,0].tolist())+" - "+sub_class+'\n')
    print(lc_index)
file.close()
    

    


Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s50  0004  -2.727889e+02   1.677561e-07 
Runtime:     00s50
Optimization status: Converged

Optimization restart 1/5, f = -272.78885953084887
Optimization restart 2/5, f = -272.78885953771714
Optimization restart 3/5, f = -272.78885953851386
Optimization restart 4/5, f = -272.78885957978355
Optimization restart 5/5, f = -272.7888595720801
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s16  0002   1.195499e+02   1.757966e-12 
Runtime:     00s16
Optimization status: Converged

Optimization restart 1/5, f = 119.5498850725433
Optimization restart 2/5, f = 119.54988507242507
Optimization restart 3/5, f = 123.2405296875848
Optimization restart 4/5, f = 119.54988507172499
Optimization restart 5/5, f = 119.5498850740907
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s72  0004  -4.431730e+02   1.26

Optimization restart 5/5, f = -969.7697357883251
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0002   2.538476e+02   1.461616e+02 
    01s34  0018   2.391755e+02   4.848940e-10 
Runtime:     01s34
Optimization status: Converged

Optimization restart 1/5, f = 239.1755185917907
Optimization restart 2/5, f = 239.1755186380284
Optimization restart 3/5, f = 239.1755186012441
Optimization restart 4/5, f = 239.1755186187596
Optimization restart 5/5, f = 239.175518591613
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s08  0001  -6.270900e+02   1.435058e-07 
    00s31  0004  -6.270900e+02   3.038195e-10 
Runtime:     00s31
Optimization status: Converged

Optimization restart 1/5, f = -627.0899696670317
Optimization restart 2/5, f = -627.0899697360437
Optimization restart 3/5, f = -627.0899677626821
Optimization restart 4/5, f = -627.0899697405655
Optimization restart 5/5, f = -627.0899697

Optimization restart 1/5, f = -759.0325748712548
Optimization restart 2/5, f = -759.032574872216
Optimization restart 3/5, f = -759.0325748660741
Optimization restart 4/5, f = -759.0325748644188
Optimization restart 5/5, f = -759.0325748604794
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s00  0000  -5.225414e+02   2.275164e-10 
    00s31  0004  -5.225414e+02   7.270911e-11 
Runtime:     00s31
Optimization status: Converged

Optimization restart 1/5, f = -522.5414333718256
Optimization restart 2/5, f = -522.5414333925416
Optimization restart 3/5, f = -522.5414333711494
Optimization restart 4/5, f = -522.5414313255787
Optimization restart 5/5, f = -522.5414333679857
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s10  0001   2.962214e+02   4.704799e-08 
    01s83  0022   2.962214e+02   4.704799e-08 
Runtime:     01s83
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimiza

 /usr/local/lib/python3.5/dist-packages/GPy/kern/src/rbf.py:43: RuntimeWarning:overflow encountered in square


Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0002  -3.823516e+02   1.229567e-12 
Runtime:     00s15
Optimization status: Converged

Optimization restart 1/5, f = -382.3516324106199
Optimization restart 2/5, f = -382.35163241200985
Optimization restart 3/5, f = -382.3516324136813
Optimization restart 4/5, f = -382.3516324116006
Optimization restart 5/5, f = -382.35163241314876
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0004  -5.956768e+02   3.463736e-03 
    00s59  0014  -5.956768e+02   4.295110e-06 
Runtime:     00s59
Optimization status: Converged

Optimization restart 1/5, f = -595.7075111763593
Optimization restart 2/5, f = -592.8222364387548
Optimization restart 3/5, f = -592.8222364133203
Optimization restart 4/5, f = -592.8222362735187
Optimization restart 5/5, f = -592.8222363086952
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |

...

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s27  0004  -1.169906e+03   1.800038e-09 
Runtime:     00s27
Optimization status: Converged

Optimization restart 1/5, f = -1169.905855077955
Optimization restart 2/5, f = -1169.9058551260982
Optimization restart 3/5, f = -1169.9058550488267
Optimization restart 4/5, f = -1169.9058550549107
Optimization restart 5/5, f = -1169.9058551083067
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s12  0002   6.071308e+02   6.459767e-13 
Runtime:     00s12
Optimization status: Converged

Optimization restart 1/5, f = 607.1308199933687
Optimization restart 2/5, f = 607.1308199985901
Optimization restart 3/5, f = 607.1308199871496
Optimization restart 4/5, f = 607.1308200032911
Optimization restart 5/5, f = 607.1308200064702
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s11  0002  -7.368175e+02   8.4735

Optimization restart 4/5, f = -359.2174704654618
Optimization restart 5/5, f = -359.2174704672319
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s14  0002  -1.479535e+02   8.237133e-11 
Runtime:     00s14
Optimization status: Converged

Optimization restart 1/5, f = -147.9534774064564
Optimization restart 2/5, f = -147.64837611059602
Optimization restart 3/5, f = -147.95347944614332
Optimization restart 4/5, f = -147.648375894355
Optimization restart 5/5, f = -147.6483759785233
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s27  0005   7.169804e+02   4.110281e-09 
Runtime:     00s27
Optimization status: Converged

Optimization restart 1/5, f = 716.9803637702848
Optimization restart 2/5, f = 712.1068273968244
Optimization restart 3/5, f = 712.1068273183878
Optimization restart 4/5, f = 735.908695869708
Optimization restart 5/5, f = 712.106827153108
Running L-BFGS-B (Scipy implementation

    00s27  0004  -5.422352e+02   5.347861e-08 
Runtime:     00s27
Optimization status: Converged

Optimization restart 1/5, f = -542.2351624666292
Optimization restart 2/5, f = -542.2351624643045
Optimization restart 3/5, f = -542.235162453952
Optimization restart 4/5, f = -542.2351624775508
Optimization restart 5/5, f = -542.2351624674152
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s06  0001  -8.003446e+02   2.970905e-08 
    00s19  0003  -8.003446e+02   6.927149e-09 
    00s62  0011  -8.003446e+02   2.971188e-08 
Runtime:     00s62
Optimization status: Converged

Optimization restart 1/5, f = -800.3446035210604
Optimization restart 2/5, f = -800.3446035000779
Optimization restart 3/5, f = -800.3446035376796
Optimization restart 4/5, f = -800.3446034941574
Optimization restart 5/5, f = -800.3446035505608
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s14  0002  -4.111964e+02   2.23

Optimization restart 5/5, f = -244.78563096835842
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s08  0001   1.830454e+03   2.247353e-06 
    02s02  0022   1.830454e+03   2.247353e-06 
Runtime:     02s02
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = 1830.4540794725744
Optimization restart 2/5, f = 1829.0436697840541
Optimization restart 3/5, f = 1829.043668100393
Optimization restart 4/5, f = 1830.454081881866
Optimization restart 5/5, f = 1829.043670453384
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0002  -7.011635e+02   1.323714e+02 
    01s66  0022  -7.116844e+02   1.381801e-07 
Runtime:     01s66
Optimization status: Converged

Optimization restart 1/5, f = -711.6843719966232
Optimization restart 2/5, f = -711.6843713264793
Optimization restart 3/5, f = -708.2568030228072
Optimization restart 4/5, f = -711.6843716545918
Optimizat

    00s31  0004  -8.925575e+02   3.004460e-13 
Runtime:     00s31
Optimization status: Converged

Optimization restart 1/5, f = -892.557456479542
Optimization restart 2/5, f = -892.5574563222958
Optimization restart 3/5, f = -892.5574562323791
Optimization restart 4/5, f = -890.5137415113227
Optimization restart 5/5, f = -890.5137409636295
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0002  -9.289667e+01   2.470058e+03 
    02s07  0022  -1.055900e+02   5.365793e-10 
Runtime:     02s07
Optimization status: Errorb'ABNORMAL_TERMINATION_IN_LNSRCH'

Optimization restart 1/5, f = -105.59004175430368
Optimization restart 2/5, f = -105.59004177832799
Optimization restart 3/5, f = -105.59004178625423
Optimization restart 4/5, f = -107.45354280567676
Optimization restart 5/5, f = -105.59004178683017
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s15  0002  -5.410806e+02   6.059520e+01 
   

In [6]:
print("Finished")

Finished
